## **1. Import Librairies:**

In [1]:
import pandas as pd
import requests

## **2. Load Datasets:**

In [2]:
timetables = pd.read_csv('timetables.csv', sep="\t")
routes = pd.read_csv('data_sncf/routes.csv', sep=",")

### **a. Timetable:**

In [3]:
timetables

,trip_id,trajet,duree
0,OCESN003100F140147152,Gare de Le Havre - Gare de Paris-St-Lazare,138
1,OCESN003190F040047309,Gare de Dieppe - Gare de Paris-St-Lazare,145
2,OCESN003198F030037315,Gare de Paris-St-Lazare - Gare de Rouen-Rive-D...,97
3,OCESN003300F030037323,Gare de Cherbourg - Gare de Paris-St-Lazare,194
4,OCESN003313F380387526,Gare de Caen - Gare de Paris-St-Lazare,149
...,...,...,...
1570,OCESN895822F0500552575,Gare de Belfort-Ville - Gare de Lyon-Perrache,244
1571,OCESN895830F0200252600,Gare de Lons-le-Saunier - Gare de Lyon-Perrache,103
1572,OCESN895880F0500552634,Gare de Belfort-Ville - Gare de Lons-le-Saunier,144
1573,OCESN895940F0200252654,Gare de Besançon-Viotte - Gare de Lons-le-Saunier,89


### **b. Routes:** 

In [4]:
routes

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,OCE1506035,OCESN,NaN,Paris-Vernon-Rouen-Le Havre,NaN,2,NaN,NaN,NaN
1,OCE1506037,OCESN,NaN,Paris-Caen-Cherbourg/Trouville,NaN,2,NaN,NaN,NaN
2,OCE1506036,OCESN,NaN,Paris-Granville,NaN,2,NaN,NaN,NaN
3,OCE1209987,OCESN,NaN,Paris-Bourges-Montluçon,NaN,2,NaN,NaN,NaN
4,OCE1417992,OCESN,NaN,Paris - Nevers,NaN,2,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
707,OCESN-87743005-87718833,OCESN,NaN,Bourg-en-Bresse - Mouchard,NaN,2,NaN,NaN,NaN
708,OCEZW-87576025-87576215,OCESN,NaN,Romorantin-Blanc-Argent - Valençay,NaN,2,NaN,NaN,NaN
709,OCEZW-87576025-87543165,OCESN,NaN,Romorantin-Blanc-Argent - Salbris,NaN,2,NaN,NaN,NaN
710,OCEZW-87543165-87576215,OCESN,NaN,Salbris - Valençay,NaN,2,NaN,NaN,NaN


## **3. Normalization:**

### **a. routes:**

In [5]:
routes.drop(columns=['route_short_name', 'route_desc', 'route_url', 'route_color', 'route_text_color'])

,route_id,agency_id,route_long_name,route_type
0,OCE1506035,OCESN,Paris-Vernon-Rouen-Le Havre,2
1,OCE1506037,OCESN,Paris-Caen-Cherbourg/Trouville,2
2,OCE1506036,OCESN,Paris-Granville,2
3,OCE1209987,OCESN,Paris-Bourges-Montluçon,2
4,OCE1417992,OCESN,Paris - Nevers,2
...,...,...,...,...
707,OCESN-87743005-87718833,OCESN,Bourg-en-Bresse - Mouchard,2
708,OCEZW-87576025-87576215,OCESN,Romorantin-Blanc-Argent - Valençay,2
709,OCEZW-87576025-87543165,OCESN,Romorantin-Blanc-Argent - Salbris,2
710,OCEZW-87543165-87576215,OCESN,Salbris - Valençay,2


## **4. Train Stations Structure:**

### **a. Train Stations as Nodes:**

In [6]:
class TrainStation:
    def __init__(self, name):
        self.name = name
        self.next = None

### **b. Routes as Linked List:**

In [7]:
class Route:
    def __init__(self, route):
        self.head = TrainStation(route[0])
        self.current = self.head
        for i in range (0, len(route)):
            current = TrainStation(route[i])
            if i != 0:
                self.current.next = current
                self.current = current
            
    def __repr__(self):
        node = self.head
        nodes = []
        while node is not None:
            nodes.append(node.name)
            node = node.next
        return " -> ".join(nodes)


In [8]:
def fetch_train_station_names(route):
    if " - " not in route:    
        return route.split("-")
    
    else:
        return route.split(" - ")

routes_list = [fetch_train_station_names(route) for route in routes['route_long_name']]
routes_list[:3]

# routes_list[0]
# routes_list[16]

[['Paris', 'Vernon', 'Rouen', 'Le Havre'],
 ['Paris', 'Caen', 'Cherbourg/Trouville'],
 ['Paris', 'Granville']]

In [9]:
route_0 = Route(routes_list[0])
route_0

routes_list[16]

linked_routes = []
for route in routes_list:
    linked_routes.append(Route(route))
linked_routes[:3]

[Paris -> Vernon -> Rouen -> Le Havre,
 Paris -> Caen -> Cherbourg/Trouville,
 Paris -> Granville]

### **c. Alternative data type:**

In [24]:
routes_graph = {}

for route in routes_list:
    for i in range (0, len(route)):        
        if route[i] not in routes_graph:
            routes_graph.update({route[i]: []})
            if i != 0:
                routes_graph[route[i]].append(route[i - 1])
            if i != len(route) - 1:
                routes_graph[route[i]].append(route[i + 1])
        elif route[i - 1] not in routes_graph[route[i]] and i != 0:
            routes_graph[route[i]].append(route[i - 1])
        elif i < len(route) - 1 and route[i + 1] not in routes_graph[route[i]]:
            routes_graph[route[i]].append(route[i + 1])

print(routes_graph["Paris"])
print(routes_graph["Caen"])
print(routes_graph["Bourges"])
print(routes_graph)

['Vernon', 'Caen', 'Granville', 'Bourges', 'Nevers', 'Orléans', 'Vaug. / Dreux / Argentan / Granville', 'Le Mans', 'CAR', 'Montargis']
['Paris', 'Cherbourg/Trouville', 'Le Mans', 'Tours']
['Paris', 'Montluçon', 'St Amand CAR', 'Issoudun', 'Tours', 'Orléans', 'Nevers']
{'Paris': ['Vernon', 'Caen', 'Granville', 'Bourges', 'Nevers', 'Orléans', 'Vaug. / Dreux / Argentan / Granville', 'Le Mans', 'CAR', 'Montargis'], 'Vernon': ['Paris', 'Rouen'], 'Rouen': ['Vernon', 'Le Havre'], 'Le Havre': ['Rouen'], 'Caen': ['Paris', 'Cherbourg/Trouville', 'Le Mans', 'Tours'], 'Cherbourg/Trouville': ['Caen'], 'Granville': ['Paris'], 'Bourges': ['Paris', 'Montluçon', 'St Amand CAR', 'Issoudun', 'Tours', 'Orléans', 'Nevers'], 'Montluçon': ['Bourges', 'Guéret', 'Gueret', 'Bordeaux'], 'Nevers': ['Paris', 'Dijon', 'Decize', 'Luzy', 'Moulins', 'Paray', 'Tours', 'Orléans', 'Bourges', 'Vierzon', 'Dijon-Ville', 'Autun'], 'Orléans': ['Paris', 'Tours', 'Nevers', 'Bourges'], 'Tours': ['Orléans', 'Paray', 'Chinon CAR',

In [11]:
len(routes_graph)

814

### **d. Get all cities coordinates:**

In [12]:
cities_coordinates = []
for city in routes_graph:
#     print(city)
#     print(requests.get("https://api-adresse.data.gouv.fr/search/?q={}&type=street".format(city.lower())))
    ret = requests.get("https://api-adresse.data.gouv.fr/search/?q={}&type=municipality".format(city.lower())).json()
#     print(ret)
    if len(ret["features"]) >= 1:
#         print(ret["features"][0]["geometry"]["coordinates"])
        cities_coordinates.append({city: ret["features"][0]["geometry"]["coordinates"]})
cities_coordinates

[{'Paris': [2.347, 48.859]},
 {'Vernon': [1.494472, 49.089623]},
 {'Rouen': [1.093912, 49.440286]},
 {'Le Havre': [0.129995, 49.507345]},
 {'Caen': [-0.372291, 49.184571]},
 {'Cherbourg/Trouville': [0.598073, 49.582241]},
 {'Granville': [-1.57398, 48.83071]},
 {'Bourges': [2.397496, 47.082492]},
 {'Montluçon': [2.605118, 46.342677]},
 {'Nevers': [3.160778, 46.988708]},
 {'Orléans': [1.910933, 47.87372]},
 {'Tours': [0.694658, 47.39565]},
 {'[IC351] PARIS': [2.347, 48.859]},
 {'AMIENS': [2.29248, 49.903034]},
 {'BOULOGNE': [2.242716, 48.836976]},
 {'CALAIS': [1.866112, 50.948388]},
 {'[IC372] PARIS': [2.347, 48.859]},
 {'MAUBEUGE': [3.965341, 50.284569]},
 {'CAMBRAI': [3.240712, 50.17263]},
 {'Le Mans': [0.190947, 47.99337]},
 {'ORL': [1.910933, 47.87372]},
 {'TOURS': [0.694658, 47.39565]},
 {'Paray': [4.111077, 46.450413]},
 {'Lyon': [4.835, 45.758]},
 {'Villefranche': [0.726735, 43.424248]},
 {'La Tour de Carol': [1.882178, 42.46909]},
 {'ROMANS/BRIANCON': [6.63739, 44.896392]},
 {'Ai

In [13]:
len(cities_coordinates)

679

In [14]:
len(routes_graph) - len(cities_coordinates)

135

## **5. Ponderations:**

### **a. Angle *based* Ponderation:**

In [15]:
import math

def angle_calculation(a, b, c):
    ang = math.degrees(math.atan2(c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1], a[0]-b[0]))
    ang = ang + 360 if ang < 0 else ang
    return ang if ang < 180 else ang - 180

angle_calculation(cities_coordinates[10]["Orléans"], cities_coordinates[0]["Paris"], cities_coordinates[2]["Rouen"])

88.98753395838276

### **b. Distance *based* Ponderation:**

In [16]:
from math import radians, cos, sin, asin, sqrt

def haversine(city, node):
    lon1, lat1, lon2, lat2 = map(radians, [city[0], city[1], node[0], node[1]])

    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371
    return c * r

haversine(cities_coordinates[10]["Orléans"], cities_coordinates[0]["Paris"])

114.19546233279856

### **c. Normalize Values:**

In [17]:
ponderation_factor_angle = 1
ponderation_factor_dist = 1

def normalize_angle(angle):
    return angle * 2 / 180 * ponderation_factor_angle

def normalize_dist(dist_node, dist):
    if dist == 0:
        return 0
    return dist_node * 2 / dist / 2 * ponderation_factor_dist

print(normalize_angle(30))
print(normalize_angle(180))

print(normalize_dist(110, 300))
print(normalize_dist(0, 300))
print(normalize_dist(300, 300))
print(normalize_dist(600, 300))


0.3333333333333333
2.0
0.36666666666666664
0.0
1.0
2.0


### **d. Generate Ponderations:**

In [18]:
cities_coordinates[:3]

[{'Paris': [2.347, 48.859]},
 {'Vernon': [1.494472, 49.089623]},
 {'Rouen': [1.093912, 49.440286]}]

Get this `list` of `dict` converted to `dict` with the *attribute name* as *key*.

In [19]:
# cities_ponderates = {city: normalize_angle(angle_calculation()) + normalize_dist(haversine(dist_node, dist)) for city}

cities_normalized = {}
for city in cities_coordinates:
    city_name = list(city.keys())[0]
    cities_normalized.update({city_name: city[city_name]})
print(list(cities_normalized.items())[:5])

[('Paris', [2.347, 48.859]), ('Vernon', [1.494472, 49.089623]), ('Rouen', [1.093912, 49.440286]), ('Le Havre', [0.129995, 49.507345]), ('Caen', [-0.372291, 49.184571])]


All cities:

In [20]:
cities_ponderate = {}
dest = "Rouen"
for city in cities_coordinates:
#     print("=======================================================")
#     print("city:")
#     print(city)
    city_name = list(city.keys())[0]
    for node in routes_graph[city_name]:
#         print(cities_normalized[node][0])
#         print(city)
        if node in cities_normalized and dest in cities_normalized and city_name in city:
            dist_pond = normalize_dist(haversine(cities_normalized[node], cities_normalized[dest]), haversine(city[city_name], cities_normalized[dest]))
            angle_pond = normalize_angle(angle_calculation(cities_normalized[node], city[city_name] ,cities_normalized[dest]))
#             print(node)
#             print(dist_pond)
#             print(angle_pond)
            cities_ponderate.update({node: dist_pond+angle_pond})
# #     print("=======================================================")

print(dict(list(cities_ponderate.items())[:15]))

{'Vernon': 0.4577770830624282, 'Caen': 1.9959846362329743, 'Granville': 3.5588346468030636, 'Bourges': 1.981856547744782, 'Nevers': 2.619631528199404, 'Orléans': 1.4424579092098364, 'Le Mans': 0.6654904677506301, 'CAR': 4.299795387634834, 'Montargis': 1.5120128346769017, 'Paris': 0.8289890483283516, 'Rouen': 0.0, 'Le Havre': 0.0442179267955996, 'Cherbourg/Trouville': 2.217918954323537, 'Tours': 2.8777584643572345, 'Montluçon': 1.987192477244143}


Specific city:

In [23]:
cities_ponderate = {}
base = "Paris"
dest = "Rouen"
city_name = base

# def ponderation(base, dest, currrent_node):
for node in routes_graph[city_name]:
#     print(cities_normalized[city_name])
#     print(node)
    if node in cities_normalized and dest in cities_normalized:
#         print(node)
        dist_pond = normalize_dist(haversine(cities_normalized[node], cities_normalized[dest]), haversine(cities_normalized[city_name], cities_normalized[dest]))
        angle_pond = normalize_angle(angle_calculation(cities_normalized[node], cities_normalized[city_name] ,cities_normalized[dest]))
#         print(node)
#         print(dist_pond)
#         print(angle_pond)
        cities_ponderate.update({node: dist_pond+angle_pond})
#     print("=======================================================")

print(dict(list(cities_ponderate.items())[:15]))

{'Vernon': 2.3269546423245724, 'Caen': 2.7840794556952093, 'Granville': 3.5588346468030636, 'Bourges': 3.2057134399770377, 'Nevers': 3.2604504645341863, 'Orléans': 2.637720941567005, 'Le Mans': 3.0376870556388376, 'CAR': 6.986372955308585, 'Montargis': 2.2489567887295108}


### **e. Recursive function:** 

In [ ]:
def recursive_graph_exploration(node, dest, current_route, valide_route):
    if node == dest:
        valide_route.append(current_route)
    if node in current_route:
        current_route = []
    else:
        current_route.append(node)
        for junction in routes_grpah[node]:
            return recursive_graph_exploration(junction, dest, current_route, valide_route)